## Defining the linear progam

Decision Variables:
- $x_{i}$ represents allocation of portfolio to asset $i$.

Contraints:
- $\sum_{i} x_i = 1$
- $x_i \ge 0 \forall i$

Objective function:
$$ \max_x \sum_{j} p_j \times (x^T \mu_{j} - \lambda \times x^T \Sigma_j x)$$

Constants being used:
- $\mu_j$, $\Sigma_j$ are expected returns and covariance matrix under scenario $j$
- $p_j$ is the probability of being in scenario $j$
- $\lambda$ is a constant representing risk tolerance.

In [14]:
using JuMP, Gurobi, LinearAlgebra, CSV

In [29]:
m = Model(with_optimizer(Gurobi.Optimizer))

Academic license - for non-commercial use only


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [30]:
num_assets = 4

@variable(m, x[1:num_assets])

@constraint(m, non_neg[i= 1:num_assets], x[i] >= 0)
@constraint(m, sum(x[i] for i=1:num_assets) == 1)

x[1] + x[2] + x[3] + x[4] == 1.0

In [31]:
rets = CSV.read("data/ExpReturns.csv", header=1)

,Column1,SPY_Return,US_Corp_Return,TreasInd_Return,Cash_Return
,String,Float64,Float64,Float64,Float64
1,Normal,0.0945161,0.0580127,0.041569,0.0163151
2,Strong,0.266753,0.0226978,-0.000699308,0.0128315
3,Weak,-0.0777203,0.0933276,0.0838374,0.0197986


In [32]:
#Have to load the data.
weak_mu = convert(Vector, rets[3, 2:num_assets+1])
weak_sigma = convert(Matrix, CSV.read("data/Weak_CovMat.csv", header=1)[:, 2:num_assets+1])

normal_mu = convert(Vector, rets[1, 2:num_assets+1])
normal_sigma = convert(Matrix, CSV.read("data/Normal_CovMat.csv", header=1)[:, 2:num_assets+1])

strong_mu = convert(Vector, rets[2, 2:num_assets+1])
strong_sigma = convert(Matrix, CSV.read("data/Strong_CovMat.csv", header=1)[:, 2:num_assets+1])

4×4 Array{Float64,2}:
  0.00961561  0.00125883   -2.63236e-5    4.7724e-5 
  0.00125883  0.0010316     0.000256363   1.34748e-5
 -2.63236e-5  0.000256363   0.000120137  -9.7315e-6 
  4.7724e-5   1.34748e-5   -9.7315e-6     1.05526e-5

In [33]:
probs = CSV.read("data/theta0_Probs.csv")[1, 2:num_assets]
normal_prob = probs[1]

strong_prob = probs[2]

weak_prob = probs[3]

0.174410327907863

In [34]:
weak_mu

4-element Array{Any,1}:
 -0.07772030624304493 
  0.09332758825036234 
  0.08383736962723685 
  0.019798631701334144

In [35]:
transpose(x)*weak_mu

-0.07772030624304493 x[1] + 0.09332758825036234 x[2] + 0.08383736962723685 x[3] + 0.019798631701334144 x[4]

In [36]:
lambda = 3
@objective(m, Max, weak_prob*(transpose(x)*weak_mu - lambda * transpose(x)*weak_sigma*x) + normal_prob*(transpose(x)*normal_mu - lambda * transpose(x)*normal_sigma*x) + strong_prob*(transpose(x)*strong_mu - lambda*transpose(x)*strong_sigma*x))

-0.051054671374439996 x[1]² - 0.02523474892721457 x[1]*x[2] - 0.0008865901259563005 x[1]*x[3] + 0.0025012771280120824 x[1]*x[4] - 0.012968896464517519 x[2]² - 0.009758019022257265 x[2]*x[3] + 0.0009514309988238847 x[2]*x[4] - 0.002955378256353406 x[3]² - 0.0002493680057120969 x[3]*x[4] - 0.0006127574870888281 x[4]² + 0.08617838935838348 x[1] + 0.059722256612685584 x[2] + 0.0436151781366348 x[3] + 0.016483709550814782 x[4]

In [37]:
optimize!(m)

Academic license - for non-commercial use only
Optimize a model with 5 rows, 4 columns and 8 nonzeros
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 9e-02]
  QObjective range [5e-04, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 4 columns, 4 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 6.000e+00
 Factor NZ  : 1.000e+01
 Factor Ops : 3.000e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -4.80073383e+05  4.82335337e+05  3.25e+03 6.98e-01  9.99e+05     0s
   1  -2.94565306e+04  2.96161789e+04  9.86e+01 7.77e-08  4.45e+04     0s
   2  -1.24015134e+02  2.45237566e+02  5.15e+00 4.06e-09  2.

In [38]:
#SPY, US_Corp, TreasIndex, Cash
value.(x)

4-element Array{Float64,1}:
 0.37700330378136065  
 0.3673954986809241   
 0.25560119604902193  
 1.4886923497077307e-9

In [39]:
pwe = [0.087, 0.06, 0.04, 0.018]
transpose(value.(x))*pwe

0.06506706521859117

In [40]:
x = value.(x)
weak_prob*(transpose(x)*weak_sigma*x) + normal_prob*transpose(x)*normal_sigma*x + strong_prob*transpose(x)*strong_sigma*x

0.0045657057716343204